In [20]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/democratvsrepublicantweets/ExtractedTweets.csv
/kaggle/input/democratvsrepublicantweets/TwitterHandles.csv
/kaggle/input/liberals-vs-conservatives-on-reddit-13000-posts/file_name.csv


## Dataset utilizzato:
https://www.kaggle.com/datasets/kapastor/democratvsrepublicantweets


In [21]:
import pandas as pd
import tensorflow
import re
import string
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [22]:
data = pd.read_csv('/kaggle/input/democratvsrepublicantweets/ExtractedTweets.csv')

print('The total number of entries is ' + str(len(data)))

The total number of entries is 86460


In [23]:
data = data.sample(frac=1).reset_index().drop(columns=['index','Handle']) #, 'Title', 'Political Lean'])
data = data.dropna()
print ('The total number of entries is '+str(len(data)))
data.head(10)

The total number of entries is 86460


,Party,Tweet
0,Democrat,So Trump's plan to hold onto his power is to i...
1,Republican,"#OnThisDay in 1878, the #California St. Cable ..."
2,Republican,"As a doctor in rural Louisiana, I have seen ho..."
3,Democrat,My full interview with @MSNBC @KasieDC @kasieh...
4,Democrat,The recently released #FarmBill is yet another...
5,Democrat,"Fox News, why are you so damn quiet now? Knowi..."
6,Republican,RT @SBAList: [THREAD] Here's a list of all the...
7,Republican,The 2018 Congressional Art Competition is unde...
8,Democrat,"Thanks, Jonathan Segal for sharing your expert..."
9,Republican,"""Accreditation increases accountability while ..."


In [24]:
DEM = len(data[data.Party=='Democrat'])
REP = len(data[data.Party=='Republican'])

print('DEM = ', DEM)
print('REP = ', REP)

DEM =  42068
REP =  44392


In [25]:
features = data.Tweet.tolist()
processed_features = []

for sentence in range(0, len(features)):
    # Remove all the special characters
    processed_feature = re.sub(r'\W', ' ', str(features[sentence]))

    # remove all single characters
    processed_feature= re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_feature)

    # Remove single characters from the start
    processed_feature = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_feature) 

    # Substituting multiple spaces with single space
    processed_feature = re.sub(r'\s+', ' ', processed_feature, flags=re.I)

    # Removing prefixed 'b'
    processed_feature = re.sub(r'^b\s+', '', processed_feature)

    # Converting to Lowercase
    processed_feature = re.sub('https://t.co','',processed_feature)
    processed_feature = re.sub('https','',processed_feature)
    processed_feature = re.sub(' co ','',processed_feature)
    processed_feature = re.sub('amp','',processed_feature)

    
    processed_feature = processed_feature.lower()

    processed_features.append(processed_feature)

In [26]:
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

In [27]:
data['Tweet'] = processed_features
data['Tweet'] =data['Tweet'].apply(lambda x: remove_punct(x))
data.head(10)

,Party,Tweet
0,Democrat,so trump plan to hold onto his power is to int...
1,Republican,onthisday in the california st cable car com...
2,Republican,as doctor in rural louisiana have seen how chi...
3,Democrat,my full interview with msnbc kasiedc kasiehunt...
4,Democrat,the recently released farmbill is yet another ...
5,Democrat,fox news why are you so damn quiet now knowing...
6,Republican,rt sbalist thread here a list of all the s sen...
7,Republican,the congressional art competition is underway...
8,Democrat,thanks jonathan segal for sharing your experti...
9,Republican,accreditation increases accountability while ...


In [28]:
processed_features = data.iloc[:,1].values
labels = data.iloc[:, 0].values
vectorizer = TfidfVectorizer (max_features=2500, min_df=7, max_df=0.8)
vectorizer.fit(processed_features)
processed_features = vectorizer.transform(processed_features).toarray()

In [29]:
def preprocess_text(text):
    text = remove_punct(text)  # Apply the existing preprocessing steps
    text = text.lower()  # Convert to lowercase
    return text

# Modello

In [32]:
processed_features = data.iloc[:, 1].values
labels = data.iloc[:, 0].values

label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

processed_features = vectorizer.transform(processed_features).toarray()

X_train, X_test, y_train, y_test = train_test_split(
    processed_features, encoded_labels, test_size=0.1, random_state=0
)

model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(2500,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=8, batch_size=64)

loss, accuracy = model.evaluate(X_test, y_test)
print("Test loss:", loss)
print("Test accuracy:", accuracy)

new_text = "This is a new text to predict the label."
processed_new_text = preprocess_text(new_text)
new_text_vector = vectorizer.transform([processed_new_text]).toarray()

prediction = model.predict(new_text_vector)
predicted_label = label_encoder.inverse_transform([prediction.argmax()])
print("Predicted label:", predicted_label)


Epoch 1/8
1216/1216 [==============================] - 5s 3ms/step - loss: 0.5439 - accuracy: 0.7130
Epoch 2/8
1216/1216 [==============================] - 4s 3ms/step - loss: 0.4931 - accuracy: 0.7409
Epoch 3/8
1216/1216 [==============================] - 4s 3ms/step - loss: 0.4845 - accuracy: 0.7461
Epoch 4/8
1216/1216 [==============================] - 4s 4ms/step - loss: 0.4762 - accuracy: 0.7527
Epoch 5/8
1216/1216 [==============================] - 4s 3ms/step - loss: 0.4654 - accuracy: 0.7586
Epoch 6/8
1216/1216 [==============================] - 4s 3ms/step - loss: 0.4516 - accuracy: 0.7701
Epoch 7/8
1216/1216 [==============================] - 4s 3ms/step - loss: 0.4357 - accuracy: 0.7842
Epoch 8/8
271/271 [==============================] - 1s 2ms/step - loss: 0.4927 - accuracy: 0.7446
Test loss: 0.4927026033401489
Test accuracy: 0.7446218132972717
1/1 [==============================] - 0s 43ms/step
Predicted label: ['Democrat']


In [19]:
def predict_class(val):
    print(val)
    val = np.round(val).astype(int)
    print(val)
    if val == 0:
        return 'Democrat'
    elif val == 1:
        return 'Republican'

In [57]:
import pickle

saved_data = {
    'vectorizer': vectorizer,
    'data': data
}

with open ('vectorizer_data.pk1', 'wb') as f:
    pickle.dump(saved_data, f)

In [60]:
model.save('model.h5')